
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Uczenie adwersarialne GAN</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

## Modele generatywne
* stara się nauczyć rzeczywistego rozkładu danych $p_X(x)$ wykorzystując model z parametrami $\theta$
* pierwsze podejście maksymalizuje likelihood $$\prod_{i=1}^np_\theta(x^i)$$
dla wszystkich przykładów uczących
  * problemy z obliczalnością $p(x)$ (patrz VAE)

# Generative Adversarial Networks GAN
* wprowadza ideę __adwersarialnego__ uczenia między __generatorem__ a __dyskryminatorem__
  * stają się  adwersarzami (przeciwnikami) w generowaniu przykładów przypominających rzeczywiste
  * generator $G:Z\longrightarrow X$ stara się "oszukać" dyskryminator
  * dyskryminator $D:X\longrightarrow [0,1]$ stara się rozpoznać skąd pochodził sygnał: prawdziwy z $X$ czy wygenerowany  $G(z)$ z wylosowanego $z$
* GAN nie definiuje $p_\theta(x)$ bezpośrednio
* definicja generatora i dyskryminatora daje ostrzejsze obrazy
  * ale są też (poważne) wady

## GAN
* $G$ generuje jak najlepsze elementy ze zmiennej $z\in Z$
* $D$ rozpoznaje czy wejście pochodzi z rzeczywistych obrazów (wysoka wartość wyjścia dyskryminatora) czy jest wygenerowane (niska wartość)
* $d$ i $G$ konkurują ze sobą starając się osiągnąć swoje własne cele
* uczenie polega na
$$\min_G\max_D V(G,D)=\min_G\max_D \mathbb{E}_{x\sim p_X}
[\log D(x)]+\mathbb{E}_{z\sim p_Z}[\log(1-D(G(z)))]$$
gdzie 
  * $p_X=p_{data}$ to prawdziwy, chociaż nieznany, rozkład danych
    * $p_{data}$ to częsty opis (może nawet zwykły...)
  * $V(G,D)$ to entropia krzyżowa typowa dla binarnej klasyfikacji
* na czym polega "gra"?
  * z opunktu widzenia $D$
    * jeśli $x$ pochodzi z prawdziwych danych, to $D$ będzie maksymalizować
    * jeśli $x$ pochodzi z $G$, to $D$ będzie minimalizować wyjscie
  * z punktu widzenia $G$
    * $G$ stara się "oszukać $D$
    * $G$ stara się __maksymalizować__ wyjście $D$, gdy daną dla $D$ jest $G(z)$ dla samplowanego $z$
    * tą maksymalizację $G$ osiąga przez coraz lepsze generowanie danych

## GAN i równowaga
* jeśli $D$ i $G$ mają wystarczającą pojemność, to __równowaga__ (equilibrium) między nimi następuje gdy $p_X(x)=p_g(x)$
* $D$ zwraca wtedy zawsze $1/2$
  * generator jest już "idealny"
  * najlepsze co dyskryminator może robić, to zwracać odpowiedź losową
  * dla ustalonego $G$ optymalny dyskryminator $$D^\ast(x)=\frac{p_g(x)}{p_g(x)+p_X(x)}$$
* wstawiając $D^\ast$ do równania dostajemy Jensen-Shannon divergence $JSD(p_X\mid\mid p_g)$
  $$\begin{align}
  JSD(p_X\mid\mid p_g)&=\frac{1}{2}KL(p_X\mid\mid M)+\frac{1}{2}KL(p_g\mid\mid M)\\
  M&=\frac{1}{2}(p_X+p_g)
  \end{align}$$
  * $JSD()$ jest symetryczną i wygładzoną wersją $KL()$
  * $0\leq JSD(p\mid\mid q)\leq1$ dla logarytmu o podstawie 2 w $KL()$
* stosunek gęstości prawdopodobieństwa
$$Dr(x)=\frac{p_X(x)}{p_g(x)}=
\frac{p(x\mid y=1)}{p(x\mid y=0)}=
\frac{p(y=1\mid x)}{p(y=0\mid x)}=
\frac{D^\ast(x)}{1-D^\ast(x)}$$
pod warunkiem $p(y=0)=p(y=1)$
  * GAN __zastępuje__ problem z obliczalnością (intractability) likelihood przez wykorzystanie względnych wartości obu rozkładów
  * GAN mierzy niezgodność między generowanym a prawdziwym rozkładem danych
  * wykorzystuje do tego dyskryminator
  

## Zalety i problemy GAN
1. uczenie gradientowe
  * jeśli rozkłady $p_g$ i $p_X$ są rozłączne albo leżą w nisko-wymiarowaym manifoldzie, to istnieje idealny dyskryminator je rozróżniający
    * doświadczalnie pokazano, że tak jest często
  * gradient $D$ przekazywany do $G$ zanika
  * to utrudnia uczenie $G$
  * teoretycznie GAN ma gwarancję nauczenia się dla $D\ast$ aproksymująego $JSD()$, ale to często nie jest implementowane przez algorytm gradientowy
2. uczenie iteracyjne
  $$\min_G\max_D V(G,D)\neq\max_D\min_G V(G,D)$$
  * uczenie $D$ aż optymalne dla ustalonego $G$ jest kosztowne
  * stąd uczenie $D$ w krokach
  * schemat uczenia prowadzi do niepewności czy rozwiązywany jest problem minimax czy maximin
    * rozwiązania obu niekoniecznie są równe
3. mode collapse
  * jeden z największych problemów GAN
  * $p_X$ ($p_{data}$) jest zwykle rozkładem multi-modalnym
  * $G$ uczy się $p_g$ prostego o kilku modach z nadzieją oszukania dyskryminatora
  * $D$ odrzuca to rozwiązanie
  * $G$ przerzuca się na inne mody…
  * jak sobie z tym radzić?
    * przewidywanie w przód reakcji dyskryminatora na poprawki generatora
    * ograniczanie gradientów dyskryminatora
    * wiele generatorów dla pojedynczego dyskryminatora
4. generator i dyskryminator mogą optymalizować tą samą funkcję, ale w przeciwnych kierunkach
5. teoretyczne gwarancje są prawdziwe w przestrzeni funkcyjnej, ale implementacja to przestrzeń parametrów i warunki nie muszą być spełnione
6. problemy z ewaluacją (to algorytm nienadzorowany)
  * miary Inception, Fr\'{e}chet Inception Distance, etc.
7. GANy gorzej od autoenkoderów radzą sobie z danymi dyskretnymi
  * wykorzystanie algorytmów typu RL
8. GANy pozwalają na równoległe tworzenie wyjscia
  * w odróżnieniu od modeli takich jak PixelCNN, PixelRNN, itp.
9. GANy nie wymagają aproksymacji ELBO jak w modelu VAE
  * nie ma potrzeby zakładania rozkładów prawdopodobieństwa
10. GANy generują ostrzejsze obrazy
  * model VAE powoduje, że optymalizowany jest błąd średniokwadratowy $\|x-G(x)\|^2$, który jest składnikiem regresyjnym
  * GANy pozwalają na śledzenie obszarów o wysokiej czętotliwości